CARGAR Y VISUALIZAR DEM PARA AOI

In [1]:
# Importar librerías necesarias
import ee
import geemap
import geopandas as gpd
from shapely.geometry import mapping

# Autenticación e inicialización (modo notebook y proyecto específico)
ee.Authenticate(auth_mode='notebook')
ee.Initialize(project='ee-maperezla')

In [2]:
# Cargar shapefile local
shapefile_path = '/home/jovyan/work/Data/AOI/PNNAmacayacu.shp'
gdf = gpd.read_file(shapefile_path)

# Obtener la geometría (primer polígono del shapefile)
geometry = ee.Geometry(mapping(gdf.geometry[0]))

# Cargar y recortar imagen DEM
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
dem_clip = dem.clip(geometry)

In [3]:
# Estilos para visualización del DEM
dem_vis_params = {
    'min': 55,
    'max': 181,
    'palette': ['#00008B', '#0000CD', '#0000FF', '#00BFFF', '#ADD8E6', '#FFFFFF', '#FFFF00', '#FFA500', '#FF4500', '#8B0000']
}

# Crear mapa interactivo con geemap
Map = geemap.Map(center=[gdf.geometry[0].centroid.y, gdf.geometry[0].centroid.x], zoom=9)
Map.addLayer(dem_clip, dem_vis_params, 'DEM recortado')

In [4]:
# Convertir GeoDataFrame a ee.FeatureCollection

aoi_ee = geemap.gdf_to_ee(gdf)

# Ahora agrega el ee.FeatureCollection al mapa

aoi_vis_params = {
    'color': 'red',  # Ejemplo: color rojo para el contorno
    'width': 2,      # Ancho del borde
    'fillColor': '00000000' # Fondo transparente (ARGB, '00' para alfa transparente)
}
Map.addLayer(aoi_ee, aoi_vis_params, 'AOI shapefile (EE FeatureCollection)')


Map.addLayerControl()
Map


Map(center=[-3.439523286382691, -70.1442113276866], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
min_val = dem_clip.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=geometry,
    scale=30, # Usar la misma escala que el DEM
    maxPixels=1e9 # Aumentar si el área es muy grande
).get('elevation').getInfo()

max_val = dem_clip.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=geometry,
    scale=30,
    maxPixels=1e9
).get('elevation').getInfo()

print(f"Valores de elevación en el área: Mínimo={min_val}m, Máximo={max_val}m")

Valores de elevación en el área: Mínimo=55m, Máximo=181m
